In [1]:
# We devide y_labels into 8 bins:
# 1st bin: delta  < -0.1; 2nd bin: -0.1 <= delta < -0.05; 3rd bin: -0.05 <= delta < -0.01;4th bin: -0.01 <= delta < 0;
# 5th bin: 0 <= delta < 0.01; 6th bin: 0.01 <= delta < 0.05; 7th bin: 0.05 <= delta < 0.1; 8th bin: delta >= 0.1.

import pickle
import pandas
import numpy as np
from datetime import datetime

import keras
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.layers import Merge 
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

with open('BTC-USD-60.pkl', 'rb') as f:
    data = pickle.load(f)

#:chunk = filter_df(df_chunk, event_type='Fill')
# sort the data based on ascending-order of time
data = data.sort_values(by=['time'])

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# save the original valuse for the fulture delta calculation

data_original = data
data_original = data_original[data_original.close > 200]

#print(data_original)
close_price = np.array(data_original.close)
#print(close_price)

In [3]:
# drop time
data = data[data.close > 200]
time = np.array(data.time)
data = data.drop('time', 1)
# data.low = np.log(data.low)
# data.high = np.log(data.high)
# data.open = np.log(data.open)
# data.close = np.log(data.close)

# print(data)
# print(time)
# print(data.close)

In [4]:
# data normalize
# mean = data.mean(axis = 0)
# std = data.std(axis = 0)
# print(mean)
# print(std)
# data = (data - mean) / std
# print(data)

In [4]:
def normalize_windows(window_data):
    #normalized_data = []
    #print(len(window_data))
    window_data = (window_data / window_data[0]) - 1 
#     window0 = window_data[0]
#     for i in range(0,len(window_data)):
#         #print(window0)
#         window_data[i] = (float(window_data[i]) / float(window0)) - 1
    return window_data

In [5]:
def switch(delta,train_Y):
    if delta < -0.1:
        train_Y.append([1,0,0,0,0,0,0,0])

    elif delta >= -0.1 and delta < -0.05:
        train_Y.append([0,1,0,0,0,0,0,0])

    elif delta >= -0.05 and delta < -0.01:
        train_Y.append([0,0,1,0,0,0,0,0])

    elif delta >= -0.01 and delta < 0:
        train_Y.append([0,0,0,1,0,0,0,0])

    elif delta >= 0 and delta < 0.01:
        train_Y.append([0,0,0,0,1,0,0,0])

    elif delta >= 0.01 and delta < 0.05:
        train_Y.append([0,0,0,0,0,1,0,0])

    elif delta >= 0.05 and delta < 0.1:
        train_Y.append([0,0,0,0,0,0,1,0])

    else:
        train_Y.append([0,0,0,0,0,0,0,1])

    return train_Y

In [6]:
#We devide y_labels into 8 bins:
#1st bin: delta  < -0.1; 2nd bin: -0.1 <= delta < -0.05; 3rd bin: -0.05 <= delta < -0.01;4th bin: -0.01 <= delta < 0;
#5th bin: 0 <= delta < 0.01; 6th bin: 0.01 <= delta < 0.05; 7th bin: 0.05 <= delta < 0.1; 8th bin: delta >= 0.1.

# fc for delta: 
# train_X: the past 60 minute average close_price, dimension: (778302, 60)
# train_Y: the delta bin for next minute. Dimension: (778302, 8)
def fc_preprocess(data, close_price , time):
    A = np.array(data.open)
    B = np.array(data.close)
    C = (A + B) / 2
    
    i = 0
    train_X = []
    train_Y = []
    while i < len(data) - 60:
        if time[i + 60] - time[i] == 3600: 
            
            temp = C[i:i + 60]
            #print(temp[60])
            train_X.append(temp)
            
            delta = (close_price[i + 60] - close_price[i + 59]) / close_price[i + 59] * 100
            
            train_Y = switch(delta,train_Y)
            
        else:
            i = i + 60
    
    return train_X, train_Y

In [7]:
train_X, train_Y = fc_preprocess(data, close_price, time)
x_tmp = np.array(train_X)
y_tmp = np.array(train_Y)
print(x_tmp.shape)
print(y_tmp.shape)


(778302, 60)
(778302, 8)


In [8]:
np.savetxt("train_X_normalized_fc_delta.csv", train_X, delimiter = ",")
np.savetxt("train_Y_normalized_fc_delta.csv", train_Y, delimiter = ",")

In [6]:
#lstm for delta: 
# train_X: the past 60 minute information(low, high, open, close, volume), dimension: (778302, 60, 5)
# train_Y: the delta bin for next minute. Dimension: (778302, 8)
def lstm_preprocess(data, close_price, time):
    window_length = 60
    A = np.array(data.low)
    B = np.array(data.high)
    C = np.array(data.open)
    D = np.array(data.close)
    E = np.array(data.volume)
    i = 0
    train_X = []
    train_Y = []
    test_X = []
    test_Y = []
    error = 0
    while i < len(C) - window_length:
         
        if time[i+window_length] - time[i] == window_length*60:
            temp = []
            normalized_temp_A = normalize_windows(np.copy(A[i:i+window_length]))
            normalized_temp_B = normalize_windows(np.copy(B[i:i+window_length]))
            normalized_temp_C = normalize_windows(np.copy(C[i:i+window_length]))
            normalized_temp_D = normalize_windows(np.copy(D[i:i+window_length]))
            normalized_temp_E = normalize_windows(np.copy(E[i:i+window_length]))
            #print(len(normalized_temp_D))
            temp.append(normalized_temp_A)
            temp.append(normalized_temp_B)
            temp.append(normalized_temp_C)
            temp.append(normalized_temp_D)
            temp.append(normalized_temp_E)

            delta = (close_price[i + 60] - close_price[i + 59]) / close_price[i + 59] * 100
            if i%5 == 0 :
                test_X.append(temp)            
                # like switch command in C, decide which bucket we should label.
                test_Y = switch(delta,test_Y)
            else:
                train_X.append(temp)                       
                # like switch command in C, decide which bucket we should label.
                train_Y = switch(delta,train_Y)
                
            i += 1
        else:
            i = i + 60

    return train_X, train_Y, test_X, test_Y

In [7]:
train_X, train_Y, test_X, test_Y = lstm_preprocess(data, close_price, time)
train_X = np.transpose(train_X, (0,2,1))
test_X = np.transpose(test_X, (0,2,1))
# for i in range(train_X.shape[2]):
#     ss = StandardScaler()
#     train_X[:,:,i] = ss.fit_transform(train_X[:,:,i])
# for i in range(test_X.shape[2]):
#     ss = StandardScaler()
#     test_X[:,:,i] = ss.fit_transform(test_X[:,:,i])


In [8]:
# x_tmp = np.array(train_X)
# y_tmp = np.array(train_Y)

print(np.shape(train_X),np.shape(train_Y),np.shape(test_X),np.shape(test_Y))


(622641, 60, 5) (622641, 8) (155661, 60, 5) (155661, 8)


In [9]:
np.save("train_X_normalized_lstm_delta.pkl", train_X)
np.save("train_Y_normalized_lstm_delta.pkl", train_Y)
np.save("test_X_normalized_lstm_delta.pkl", test_X)
np.save("test_Y_normalized_lstm_delta.pkl", test_Y)

In [12]:
# baseline for delta: calculate the average delta in the past 60 minutes, and use this value for the prediction for next
# minute
# train_X: the past 60 minute close_price, dimension: (778302, 60)
# train_Y: the delta bin for next minute. Dimension: (778302, 8)
def baseline_preprocess(data, close_price , time):
    
    C = np.array(data.close)
  
    i = 0
    train_X = []
    train_Y = []
    while i < len(data) - 60:
        if time[i + 60] - time[i] == 3600: 
            
            temp = close_price[i:i + 60]
            #print(temp[60])
            train_X.append(temp)
            
           # delta = (close_price[i + 59] - close_price[i]) / close_price[i] * 100 

           # delta = delta / 60
            delta = (close_price[i + 60] - close_price[i + 59]) / close_price[i + 59] * 100 
            if delta < -0.1:
                
                train_Y.append([1,0,0,0,0,0,0,0])
                i = i + 1
            elif delta >= -0.1 and delta < -0.05:
                
                train_Y.append([0,1,0,0,0,0,0,0])
                i = i + 1
            elif delta >= -0.05 and delta < -0.01:
                
                train_Y.append([0,0,1,0,0,0,0,0])
                i = i + 1
            elif delta >= -0.01 and delta < 0:
                train_Y.append([0,0,0,1,0,0,0,0])
                i = i + 1
            elif delta >= 0 and delta < 0.01:
                train_Y.append([0,0,0,0,1,0,0,0])
                i = i + 1   
            elif delta >= 0.01 and delta < 0.05:
                train_Y.append([0,0,0,0,0,1,0,0])
                i = i + 1 
            elif delta >= 0.05 and delta < 0.1:
                train_Y.append([0,0,0,0,0,0,1,0])
                i = i + 1 
            else:
                train_Y.append([0,0,0,0,0,0,0,1])
                i = i + 1
        else:
            i = i + 60
    
    return train_X, train_Y

In [13]:
train_X, train_Y = baseline_preprocess(data, close_price, time)
x_tmp = np.array(train_X)
y_tmp = np.array(train_Y)
print(x_tmp.shape)
print(y_tmp.shape)

(778302, 60)
(778302, 8)


In [14]:
np.savetxt("train_X_normalized_baseline_delta.csv", train_X, delimiter = ",")
np.savetxt("train_Y_normalized_baseline_delta.csv", train_Y, delimiter = ",")

In [ ]:
# check the distribution of data in buckets
train_X, train_Y = lstm_preprocess(data, close_price, time)
train_X = np.transpose(train_X, (0,2,1))
for i in range(train_X.shape[2]):
    ss = StandardScaler()
    train_X[:,:,i] = ss.fit_transform(train_X[:,:,i])
x_tmp = np.array(train_X)
y_tmp = np.array(train_Y)
print(x_tmp.shape)
print(y_tmp.shape)